In [1]:
import pickle
import pandas as pd
import torch
import torch.nn as nn

In [2]:
competition_test_cache = 'cache/competition-test2.pickle'
metadata_cache = 'cache/metadata2.pickle'

In [3]:
competition_test = pd.read_pickle(competition_test_cache)
with open(metadata_cache, 'rb') as handle:
    metadata = pickle.load(handle)

In [4]:
def process_features(df):
    return torch.tensor(df[['QUARTER_HOUR','DAY_OF_WEEK','WEEK_OF_YEAR','ORIGIN_CALL_ENCODED','TAXI_ID_ENCODED','ORIGIN_STAND_ENCODED',
                           'dt_2014-08-14 00:00:00','dt_2014-09-30 00:00:00','dt_2014-10-06 00:00:00','dt_2014-11-01 00:00:00',
                           'dt_2014-12-21 00:00:00']].values)

In [5]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.embed_quarter_hour = nn.Embedding(metadata['n_quarter_hours'], 10)
        self.embed_day_of_week = nn.Embedding(metadata['n_days_per_week'], 10)
        self.embed_week_of_year = nn.Embedding(metadata['n_weeks_per_year'],10)
        self.embed_client_ids = nn.Embedding(metadata['n_client_ids'],10)
        self.embed_taxi_ids = nn.Embedding(metadata['n_taxi_ids'],10)
        self.embed_stand_ids = nn.Embedding(metadata['n_stand_ids'],10)
        self.layer1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(in_features=65, out_features=160),
            nn.ReLU()
        )
        self.layer2 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(in_features = 160, out_features = 100),
            nn.ReLU()
        )
        self.layer3 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(in_features = 100, out_features = 50),
            nn.ReLU()
        )
        self.layer4 = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(50,15),
            nn.ReLU()
        )
        self.output_layer = nn.Linear(in_features=15, out_features=1)

    def forward(self, x):
        qhr = self.embed_quarter_hour(x[:,0].to(torch.int32))
        dow = self.embed_day_of_week(x[:,1].to(torch.int32))
        woy = self.embed_week_of_year(x[:,2].to(torch.int32))
        ci = self.embed_client_ids(x[:,3].to(torch.int32))
        ti = self.embed_taxi_ids(x[:,4].to(torch.int32))
        si =  self.embed_stand_ids(x[:,5].to(torch.int32))
        x = torch.cat([qhr,dow,woy,ci,ti,si,x[:,6:]],axis=1)
        x = x.to(torch.float32)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.output_layer(x)
        return x

In [6]:
model1 = MLP()

In [ ]:
model1.load_state_dict(torch.load('model_state/model_epoch199_LSTM_5Layer.pth'))
model1.eval()

In [7]:
class LSTMNet(nn.Module):
    def __init__(self):
        super(LSTMNet, self).__init__()
        self.embed_quarter_hour = nn.Embedding(metadata['n_quarter_hours'], 10)
        self.embed_day_of_week = nn.Embedding(metadata['n_days_per_week'], 10)
        self.embed_week_of_year = nn.Embedding(metadata['n_weeks_per_year'],10)
        self.embed_client_ids = nn.Embedding(metadata['n_client_ids'],10)
        self.embed_taxi_ids = nn.Embedding(metadata['n_taxi_ids'],10)
        self.embed_stand_ids = nn.Embedding(metadata['n_stand_ids'],10)

        self.lstm = nn.LSTM(input_size=65, hidden_size=180, num_layers=5, batch_first=True, dropout=0.5)
        self.fc = nn.Linear(180, 1)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        qhr = self.embed_quarter_hour(x[:,0].to(torch.int32))
        dow = self.embed_day_of_week(x[:,1].to(torch.int32))
        woy = self.embed_week_of_year(x[:,2].to(torch.int32))
        ci = self.embed_client_ids(x[:,3].to(torch.int32))
        ti = self.embed_taxi_ids(x[:,4].to(torch.int32))
        si = self.embed_stand_ids(x[:,5].to(torch.int32))
        x = torch.cat([qhr,dow,woy,ci,ti,si,x[:,6:]],axis=1)
        x = x.to(torch.float32)

        # Reshape input data to (batch_size, sequence_length, input_size) for LSTM
        x = x.view(x.size(0), 1, -1)

        x, _ = self.lstm(x)
        x = self.relu(x)
        x = self.fc(x[:, -1, :])

        return x

In [8]:
model2 = LSTMNet()

In [ ]:
model2.load_state_dict('model_state/model_moreunit_epoch90.pth')
model2.eval()

In [ ]:
predict_input = torch.tensor(process_features(competition_test))

In [ ]:
def predict(model):
    predict_output = model(predict_input)
    return predict_output

out1 = predict(model1)
out2 = predict(model2)

In [ ]:
predict1 = pd.read_csv('test_public.csv')
predict1 = predict1['TRIP_ID']
predict_tensor1 = out1.to('cpu').detach().numpy().flatten()
predict1 = pd.concat([predict1, pd.DataFrame(predict_tensor1)], axis=1)
predict1 = predict1.rename(columns={0: 'TRAVEL_TIME'})
predict1

In [ ]:
predict2 = pd.read_csv('test_public.csv')
predict2 = predict2['TRIP_ID']
predict_tensor2 = out2.to('cpu').detach().numpy().flatten()
predict2 = pd.concat([predict2, pd.DataFrame(predict_tensor1)], axis=1)
predict2 = predict2.rename(columns={0: 'TRAVEL_TIME'})
predict2